NOMAD | BL-1B | SNS
===

## Introduction

The NOMAD auto-reduction routine is a wrapper around the `TotalScatteringReduction` module which is the backbone of the `mantidtotalscattering` package for neutron total scattering data reduction. In the `TotalScatteringReduction` method, different scheme can be specified for grouping detectors at both the data processing and output stage. There are two commonly used ways of grouping detectors -- group detectors by physical banks and group all detectors into a single bank. For NOMAD auto-reduction, both of the approaches are implemented. The multi-banks mode is advantageous in terms of using as much of the high-angle bank (which is with high resolution) as possible. Here, the resolution of the total scattering pattern in $d$-space can be derived as follows,

$$2d\,sin\theta = \lambda \Rightarrow \Delta d = \frac{\lambda}{2}\frac{cos\theta}{sin^2\theta} \Rightarrow \frac{\Delta d}{d} = cot\theta\tag{1}
$$

from which one can see that the resolution is increasing monotonically as the increase of the scattering angle. One of the biggest advantages of the single-bank mode is that it can directly give a single $Q$-space total scattering pattern for which the Fourier transform can be performed automatically, given necessary input parameters pre-configured.

Upon the reduction of each individual run at the auto-reduction stage, the align and focused data will be saved to a certain location (which is specific to each experiment) as cache files. Meanwhile, the individual reduction configuration will be saved as JSON file. Afterwards, the auto-reduction engine will check the output directory for existing JSON files and search (by `sample title`) for all runs that are similar to the current run. Here, by `similar` runs, we mean all those repeated runs for the same sample under the same measurement condition. Then a merged JSON file will be generated to incorporate all those similar runs, after which a `TotalScatteringReduction` run will be executed with the merged JSON file as the input. Once again, both the single-bank and multi-banks mode will be used for running the merged JSON input. After all, all those merged JSON input will be finally merged into a comprehensive JSON file, which then can be later imported into ADDIE for manual reduction purpose. In fact, given the current iterative approach (as detailed below) of running the auto-reduction routine, in most cases, manual reduction is not necessary. However, in case some improper configuration at the auto-reduction stage, one can perform the manual reduction using ADDIE (as detailed here) via interfacing with the generated JSON file.

Given the proper background subtraction and instrument effect normalization and the necessary corrections (absorption effect, multiple scattering effect, etc.) being in place, one is expecting the reduced total scattering will be sitting on an absolute scale, i.e., the data is directly comparable with a theoretical calculation of the total scattering pattern upon a structure model consistent with the measured sample, without the need for an extra scale factor. In such a situation, the reduced $S(Q)$ data [as defined below in Eqn. (2)] should asymptotically approach the self-scattering level [see Eqn. (3) below] in the high-$Q$ region.

$$S(Q) = \frac{1}{N}\sum_{j,k}b_jb_k\frac{sin(Qr_{jk})}{Qr_{jk}}\tag{2}$$

$$\langle b^2\rangle = \sum_ic_ib_i^2\tag{3}$$

where $c_j$, $c_k$ refers to the number concentration of atom species $j$ and $k$, respectively. $b_j$ and $b_k$ is for the scattering length of each species, and $N$ is the total number of atoms in the material. However, in practice, it is almost always the case that even the mass density is measured correctly and all the necessary corrections are in place, the high-$Q$ level will be slightly off the self-scattering level. With this regard, the packing fraction tweaking factor is introduced for adjusting the high-$Q$ level towards the self-scattering level -- equivalently, one can also imagine this as tweaking the effective density of the material. At this point, one may wonder why we do not directly scale the data to the self-scattering level by simply multiplying a scale factor, since anyhow tweaking the packing fraction seems to be doing exactly the same thing. However, this is not true, since tweaking the packing fraction will not only induce a scale factor but also will induce a non-uniform impact on the absorption correction across the $Q$-space. This is also the reason why one has to re-do the absorption correction any time the packing fraction is tweaked. So, the implementation here is that after each auto-reduction run, we will be checking the high-$Q$ level of the output $S(Q)$ pattern, compare it to the self-scattering level and suggest a new packing fraction. Such a procedure will continue until the packing fraction value is converged. Usually, we found that the packing fraction value will converge after 2 or 3 cycles.

> N.B. The absorption calculation (no matter which approach is used, such as numerical integration or Monte Carlo ray tracing) is computationally heavy. Therefore, if staying with the pixel-by-pixel level of correction for the absorption effect, the iterative manner of reducing the total scattering data will make the auto-reduction running time unrealistically long. Concerning this, we implement a routine for performing the absorption calcualtion in a group manner -- see [here](https://powder.ornl.gov/total_scattering/data_reduction/mts_abs_ms.html#performance-boost) for relevant discussion about the performance boost in the case of absorption correction.

## Implementation Flow

<img src="../imgs/NOM_AutoRed.svg" alt="drawing" width="1000"/>

## How-to

0. Normally, during the experiment, when data has been collected and saved to the analysis cluster, the corresponding autoreduction will be kicked off automatically and the produced data will be saved to, `/SNS/NOM/shared/autoreduce`. In case one needs to redo the autoreduction (say, when some pre-configuration was wrong or problematic, etc.), one could follow the steps below to re-do the autoreduction.

1. Go to NOMAD monitor, <a href="https://monitor.sns.gov/report/nom/" target="_blank">https://monitor.sns.gov/report/nom/</a>.

2. There you will a list of IPTS's -- select the one you are interested in and have access to.

    <img src="../imgs/nom_auto_1.png" alt="drawing" width="800"/>

3. Click into an IPTS and you will see the list of runs,

    <img src="../imgs/nom_auto_2.png" alt="drawing" width="800"/>
    
4. Scroll to the botom of the page, and find the `reduction` option,

    <img src="../imgs/nom_auto_3.png" alt="drawing" width="800"/>

5. Click on the link and the autoreduction will be kicked off.

6. For users with admin access, one can go to the top of the page and click on `admin` to see,

    <img src="../imgs/nom_auto_4.png" alt="drawing" width="800"/>
    
    where one can fill in necessary information and selection `REDUCTION.REQUEST` option from the dropdown menu to submit batch runs of autoreduction jobs.

## Coder Notes

1. The autoreduction for various techniques at SNS and HFIR diffractometers have been re-vamped so that a consistent scheme is applied across techniques in terms of implementation. The design is to use the suitable conda environment for performing the reduction job, where the conda environment being used can be easily configured within the autoreduction script (as simple as specifying, e.g.,  `CONDA_ENV = 'mantid-dev'`).

2. To test the autoreduction routine locally, one could grab the autoreduction script, make changes, and test it with the command,

    2.1. `. /opt/anaconda/etc/profile.d/conda.sh && conda activate mantidtotalscattering`
    
    2.2. `python /path/to/reduce_NOM.py /path/to/input/nexus /path/to/output/that/is/out/of/the/way`